<img src = "https://drive.google.com/uc?id=13Un3zyjk1feC8ryZvIXe-kyvOIf5FP5h">


# Loading the NIDS from Google Drive
## Download the model [here](https://drive.google.com/drive/folders/1-5WKAhP9SIDy0ZHDBV0_c1ErX0rp1aFO?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
model500Path = "hidden path"
model = tf.saved_model.load(model500Path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading the class "Attack" of NSL-KDD
## Download [here](https://drive.google.com/file/d/1M-SmOcRmjQGh_5BUP6F-Fhg_Mr_zXO3q/view?usp=sharing)

In [ ]:
import numpy as np
Attacks = 'hidden path'
Attacks = np.load(Attacks)
Attacks = np.reshape(Attacks, (Attacks.shape[0], 113))


# Boundary algorithm

In [ ]:
from typing import TYPE_CHECKING, Callable, List, Optional, Tuple, Union

    #Convert an array of labels to binary class matrix.
    #:param labels: An array of integer labels of shape `(nb_samples,)`.
    #:param nb_classes: The number of classes (possible labels).
    #:return: A binary matrix representation of `y` in the shape `(nb_samples, nb_classes)`.
def to_categorical(labels: Union[np.ndarray, List[float]], nb_classes: Optional[int] = None) -> np.ndarray:
  labels = np.full((labels.shape[0]), labels)
  #labels = np.array(labels, dtype=int)
  if nb_classes is None:
    nb_classes = np.max(labels) + 1
  categorical = np.zeros((labels.shape[0], nb_classes), dtype=np.float32)
  categorical[np.arange(labels.shape[0]), np.squeeze(labels)] = 1
  return categorical



def check_and_transform_label_format( labels: np.ndarray, nb_classes: Optional[int] = None, return_one_hot: bool = True
) -> np.ndarray:
  labels_return = labels
  if type(nb_classes) != int :
    nb_classes = nb_classes[0]
  labels_return = to_categorical(labels, nb_classes)
  return labels_return 


In [ ]:
class classifier:
  model = model
  def __init__(self, model, nb_classes, clip_values):
    self.model = model,
    self.nb_classes = 2,
    self.clip_values = None,
    self.input_shape = (113,)
  def predict(self, x, batch_size):
    if batch_size == 1:
      inp = tf.convert_to_tensor(x)
      inp = tf.cast(inp, tf.float32)
      inp = tf.reshape(inp, ( 1, 1, 113))
      labeling = model(inp)
      label = np.argmax(labeling)
      return label
    else:
      x_all = x
      labels = np.zeros((batch_size), int)
      for i in range(batch_size):
        x = x_all[i]
        inp = tf.convert_to_tensor(x)
        inp = tf.cast(inp, tf.float32)
        inp = tf.reshape(inp, ( 1, 1, 113))
        labeling = model(inp)
        label = np.argmax(labeling)
        labels[i] = label
      
      return labels



In [ ]:
from tqdm.auto import tqdm, trange
ART_NUMPY_DTYPE = np.float32
class BoundaryAttack():
    """
    Implementation of the boundary attack from Brendel et al. (2018). This is a powerful black-box attack that
    only requires final class prediction.

    | Paper link: https://arxiv.org/abs/1712.04248
    """

    attack_params =  [
        "targeted",
        "delta",
        "epsilon",
        "step_adapt",
        "max_iter",
        "num_trial",
        "sample_size",
        "init_size",
        "batch_size",
        "verbose",
    ]
    def __init__(self, estimator, batch_size, targeted, delta, epsilon, step_adapt, max_iter, num_trial, sample_size, init_size, min_epsilon , verbose):
      self.estimator = estimator
      self.batch_size = batch_size
      self.targeted = targeted
      self.delta = delta
      self.epsilon = epsilon
      self.step_adapt = step_adapt
      self.max_iter = max_iter
      self.num_trial = num_trial
      self.sample_size = sample_size
      self.init_size = init_size
      self.min_epsilon = min_epsilon
      self.batch_size = batch_size
      self.verbose = verbose
      
      #self._check_params()

      self.curr_adv: Optional[np.ndarray] = None

    def generate(self, x: np.ndarray, y: Optional[np.ndarray] = None, **kwargs) -> np.ndarray:
        """
        Generate adversarial samples and return them in an array.

        :param x: An array with the original inputs to be attacked.
        :param y: Target values (class labels) one-hot-encoded of shape (nb_samples, nb_classes) or indices of shape
                  (nb_samples,). If `self.targeted` is true, then `y` represents the target labels.
        :param x_adv_init: Initial array to act as initial adversarial examples. Same shape as `x`.
        :type x_adv_init: `np.ndarray`
        :return: An array holding the adversarial examples.

        """
        clip_min, clip_max = np.min(x), np.max(x)
        if y is None:
            # Throw error if attack is targeted, but no targets are provided
            if self.targeted:  # pragma: no cover
                raise ValueError("Target labels `y` need to be provided for a targeted attack.")

            # Use model predictions as correct outputs
            y = get_labels_np_array(self.estimator.predict(x,self.batch_size ))  # type: ignore
        y = check_and_transform_label_format(y, self.estimator.nb_classes, return_one_hot=False)
        if y is not None and self.estimator.nb_classes == 2 and y.shape[1] == 1:
            raise ValueError(  # pragma: no cover
                "This attack has not yet been tested for binary classification with a single output classifier."
            )

        # Get clip_min and clip_max from the classifier or infer them from data
        #if self.estimator.clip_values is not None:
            #clip_min, clip_max = self.estimator.clip_values
        #else:
            #clip_min, clip_max = np.min(x), np.max(x)

        # Prediction from the original images
        preds = self.estimator.predict(x, self.batch_size)
        preds = check_and_transform_label_format(preds, self.estimator.nb_classes, return_one_hot=True)

        # Prediction from the initial adversarial examples if not None
        x_adv_init = kwargs.get("x_adv_init")

        if x_adv_init is not None:
            init_preds = np.argmax(self.estimator.predict(x_adv_init, batch_size=self.batch_size), axis=1)
        else:
            init_preds = [None] * len(x)
            x_adv_init = [None] * len(x)

        # Assert that, if attack is targeted, y is provided
        if self.targeted and y is None:  # pragma: no cover
            raise ValueError("Target labels `y` need to be provided for a targeted attack.")

        # Some initial setups
        x_adv = x.astype(ART_NUMPY_DTYPE)
  
        # Generate the adversarial samples
        for ind, val in enumerate(tqdm(x_adv, desc="Boundary attack", disable=not self.verbose)):
            if self.targeted:
                x_adv[ind] = self._perturb(
                    x=val,
                    y=np.argmax(y[ind]),
                    y_p=np.argmax(preds[ind]),
                    init_pred=init_preds[ind],
                    adv_init=x_adv_init[ind],
                    clip_min=clip_min,
                    clip_max=clip_max,
                )
            else:
                x_adv[ind] = self._perturb(
                    x=val,
                    y=-1,
                    y_p=np.argmax(preds[ind]),
                    init_pred=init_preds[ind],
                    adv_init=x_adv_init[ind],
                    clip_min=clip_min,
                    clip_max=clip_max,
                )
 

        return x_adv
    def _perturb(
        self,
        x: np.ndarray,
        y: int,
        y_p: int,
        init_pred: int,
        adv_init: np.ndarray,
        clip_min: float,
        clip_max: float,
    ) -> np.ndarray:
        """
        Internal attack function for one example.

        :param x: An array with one original input to be attacked.
        :param y: If `self.targeted` is true, then `y` represents the target label.
        :param y_p: The predicted label of x.
        :param init_pred: The predicted label of the initial image.
        :param adv_init: Initial array to act as an initial adversarial example.
        :param clip_min: Minimum value of an example.
        :param clip_max: Maximum value of an example.
        :return: An adversarial example.
        """
        # First, create an initial adversarial sample
        initial_sample = self._init_sample(x, y, y_p, init_pred, adv_init, clip_min, clip_max)
       
        # If an initial adversarial example is not found, then return the original image
        if initial_sample is None:
            return x

        # If an initial adversarial example found, then go with boundary attack
        x_adv = self._attack(
            initial_sample[0],
            x,
            y_p,
            initial_sample[1],
            self.delta,
            self.epsilon,
            clip_min,
            clip_max,
        )

        return x_adv
    def one_hot_encode(self, arr):
      if type(arr) == np.int64:
        hot = [0, 0]
        if arr == 0:
          hot[0] = 1
        else:
          hot[1] = 1
        return np.array(hot)

      res = []
      for i in range(len(arr)):
        hot = [0, 0]
        if arr[i] == 0:
          hot[0] = 1
        else:
          hot[1] = 1
        res.append(hot)
      par = np.array(res)
      return par
    def _attack(
        self,
        initial_sample: np.ndarray,
        original_sample: np.ndarray,
        y_p: int,
        target: int,
        initial_delta: float,
        initial_epsilon: float,
        clip_min: float,
        clip_max: float,
    ) -> np.ndarray:
        """
        Main function for the boundary attack.

        :param initial_sample: An initial adversarial example.
        :param original_sample: The original input.
        :param y_p: The predicted label of the original input.
        :param target: The target label.
        :param initial_delta: Initial step size for the orthogonal step.
        :param initial_epsilon: Initial step size for the step towards the target.
        :param clip_min: Minimum value of an example.
        :param clip_max: Maximum value of an example.
        :return: an adversarial example.
        """
        # Get initialization for some variables
        x_adv = initial_sample
        self.curr_delta = initial_delta
        self.curr_epsilon = initial_epsilon

        self.curr_adv = x_adv

        # Main loop to wander around the boundary
        for _ in trange(self.max_iter, desc="Boundary attack - iterations", disable=True):
            # Trust region method to adjust delta
            for _ in range(self.num_trial):
                potential_advs = []
                for _ in range(self.sample_size):
                    potential_adv = x_adv + self._orthogonal_perturb(self.curr_delta, x_adv, original_sample)
                    
                    potential_adv = np.clip(potential_adv, clip_min, clip_max)
                    potential_advs.append(potential_adv)
                   
                preds =  self.estimator.predict(np.array(potential_advs), batch_size=self.sample_size)
               
                preds = self.one_hot_encode(preds)
               
                preds = np.argmax(
                    preds, axis = 1
                   
                )


                if self.targeted:
                    satisfied = preds == target
                else:
                    satisfied = preds != y_p
               

                delta_ratio = np.mean(satisfied)

                if delta_ratio < 0.2:
                    self.curr_delta *= self.step_adapt
                elif delta_ratio > 0.5:
                    self.curr_delta /= self.step_adapt

                if delta_ratio > 0:
                    x_advs = np.array(potential_advs)[np.where(satisfied)[0]]
                    break
            else:  
                return x_adv

            # Trust region method to adjust epsilon
            for _ in range(self.num_trial):
                perturb = np.repeat(np.array([original_sample]), len(x_advs), axis=0) - x_advs
              
                perturb *= self.curr_epsilon
                potential_advs = x_advs + perturb
                potential_advs = np.clip(potential_advs, clip_min, clip_max)
                preds =  self.estimator.predict(potential_advs, batch_size=perturb.shape[0])
                preds = self.one_hot_encode(preds)
              
                if potential_advs.shape[0] > 1:
                  preds = np.argmax(
                    preds, axis = 1
                   
                  )
                else:
                  preds = np.argmax(
                    preds, axis = 0
                   
                  )



                if self.targeted:
                    satisfied = preds == target
                else:
                    satisfied = preds != y_p

                epsilon_ratio = np.mean(satisfied)

                if epsilon_ratio < 0.2:
                    self.curr_epsilon *= self.step_adapt
                elif epsilon_ratio > 0.5:
                    self.curr_epsilon /= self.step_adapt

                if epsilon_ratio > 0:
                    x_adv = self._best_adv(original_sample, potential_advs[np.where(satisfied)[0]])
                    self.curr_adv = x_adv
                    break
            else:  # pragma: no cover
                print("Adversarial example found but not optimal.")
                return self._best_adv(original_sample, x_advs)

            if self.curr_epsilon < self.min_epsilon:
                return x_adv

        return x_adv
    def _orthogonal_perturb(self, delta: float, current_sample: np.ndarray, original_sample: np.ndarray) -> np.ndarray:
        """
        Create an orthogonal perturbation.

        :param delta: Initial step size for the orthogonal step.
        :param current_sample: Current adversarial example.
        :param original_sample: The original input.
        :return: a possible perturbation.
        """
        # Generate perturbation randomly
        perturb = np.random.randn(*self.estimator.input_shape).astype(ART_NUMPY_DTYPE)

        # Rescale the perturbation
        perturb /= np.linalg.norm(perturb)
        perturb *= delta * np.linalg.norm(original_sample - current_sample)

        # Project the perturbation onto sphere
        direction = original_sample - current_sample

        direction_flat = direction.flatten()
        perturb_flat = perturb.flatten()

        direction_flat /= np.linalg.norm(direction_flat)
        perturb_flat -= np.dot(perturb_flat, direction_flat.T) * direction_flat
        perturb = perturb_flat.reshape(self.estimator.input_shape)

        hypotenuse = np.sqrt(1 + delta ** 2)
        perturb = ((1 - hypotenuse) * (current_sample - original_sample) + perturb) / hypotenuse
        return perturb
  


    def _init_sample(
        self,
        x: np.ndarray,
        y: int,
        y_p: int,
        init_pred: int,
        adv_init: np.ndarray,
        clip_min: float,
        clip_max: float,
    ) -> Optional[Tuple[np.ndarray, int]]:
        """
        Find initial adversarial example for the attack.

        :param x: An array with one original input to be attacked.
        :param y: If `self.targeted` is true, then `y` represents the target label.
        :param y_p: The predicted label of x.
        :param init_pred: The predicted label of the initial image.
        :param adv_init: Initial array to act as an initial adversarial example.
        :param clip_min: Minimum value of an example.
        :param clip_max: Maximum value of an example.
        :return: an adversarial example.
        """
        nprd = np.random.RandomState()
        initial_sample = None
        
        if self.targeted:
            # Attack satisfied
            if y == y_p:
                return None

            # Attack unsatisfied yet and the initial image satisfied
            if adv_init is not None and init_pred == y:
                return adv_init.astype(ART_NUMPY_DTYPE), init_pred

            # Attack unsatisfied yet and the initial image unsatisfied
            for _ in range(self.init_size):
                random_img = nprd.uniform(clip_min, clip_max, size=x.shape).astype(x.dtype)
                random_class = np.argmax(
                    self.estimator.predict(np.array([random_img]), batch_size=1),
                   
                )

                if random_class == y:
                    initial_sample = random_img, random_class

                    break
            else:
                print("Failed to draw a random image that is adversarial, attack failed.")

        else:
            # The initial image satisfied
            if adv_init is not None and init_pred != y_p:
                return adv_init.astype(ART_NUMPY_DTYPE), init_pred

            # The initial image unsatisfied
            for _ in range(self.init_size):
                random_img = nprd.uniform(clip_min, clip_max, size=x.shape).astype(x.dtype)
                random_class = np.argmax(
                    self.estimator.predict(np.array([random_img]), batch_size=self.batch_size),
                    axis=1,
                )[0]

                if random_class != y_p:
                    initial_sample = random_img, random_class

                    #print.info("Found initial adversarial image for untargeted attack.")
                    break
            else:  # pragma: no cover
                print("Failed to draw a random image that is adversarial, attack failed.")

        return initial_sample
    @staticmethod
    def _best_adv(original_sample: np.ndarray, potential_advs: np.ndarray) -> np.ndarray:
        """
        From the potential adversarial examples, find the one that has the minimum L2 distance from the original sample
        :param original_sample: The original input.
        :param potential_advs: Array containing the potential adversarial examples
        :return: The adversarial example that has the minimum L2 distance from the original input
        """
        shape = potential_advs.shape
        min_idx = np.linalg.norm(original_sample.flatten() - potential_advs.reshape(shape[0], -1), axis=1).argmin()
        return potential_advs[min_idx]

    def _check_params(self) -> None:
        if not isinstance(self.max_iter, int) or self.max_iter < 0:
            raise ValueError("The number of iterations must be a non-negative integer.")

        if not isinstance(self.num_trial, int) or self.num_trial < 0:
            raise ValueError("The number of trials must be a non-negative integer.")

        if not isinstance(self.sample_size, int) or self.sample_size <= 0:
            raise ValueError("The number of samples must be a positive integer.")

        if not isinstance(self.init_size, int) or self.init_size <= 0:
            raise ValueError("The number of initial trials must be a positive integer.")

        if self.epsilon <= 0:
            raise ValueError("The initial step size for the step towards the target must be positive.")

        if self.delta <= 0:
            raise ValueError("The initial step size for the orthogonal step must be positive.")

        if self.step_adapt <= 0 or self.step_adapt >= 1:
            raise ValueError("The adaptation factor must be in the range (0, 1).")

        if not isinstance(self.min_epsilon, (float, int)) or self.min_epsilon < 0:
            raise ValueError("The minimum epsilon must be non-negative.")

        if not isinstance(self.verbose, bool):
            raise ValueError("The argument `verbose` has to be of type bool.")



# AE crafting

In [ ]:
self.estimator = estimator
      self.batch_size = batch_size
      self.targeted = targeted
      self.delta = delta
      self.epsilon = epsilon
      self.step_adapt = step_adapt
      self.max_iter = max_iter
      self.num_trial = num_trial
      self.sample_size = sample_size
      self.init_size = init_size
      self.min_epsilon = min_epsilon
      self.batch_size = batch_size
      self.verbose = verbose

In [ ]:
files = ["2500:3000"]
for i in range(len(files)):
  start = 11000 + i*500
  end = 11500 +i*500
  batch = 100
  clf = classifier(model, 2, None)
  target = np.zeros((batch), int)
  crafter = BoundaryAttack(clf, batch, True, 0.01, 0.01, 0.667, 100, 25, 20, 100, 0.0, True)
  x_advr = crafter.generate(Attacks[0:100], target)


Boundary attack:   0%|          | 0/100 [00:00<?, ?it/s]

# Saving the AE in Google Drive

In [ ]:
with open('hidden path', 'w') as f:
    np.savetxt(f, x_advr)

# Attacking the NIDS

In [ ]:
success = 0
def predict(model, x):
  inp = tf.convert_to_tensor(x)
  inp = tf.cast(inp, tf.float32)
  inp = tf.reshape(inp, ( 1, 1, 113))
  labeling = model(inp)
  label = np.argmax(labeling)
  return label
for i in range(100):
  prediction = predict(model,x_advr[i])
  if prediction == 0:
    success += 1



In [ ]:
print("success rate of Boundary is ", success, "%")


success rate of Boundary is  98 %
